In [1]:
### PROJET 3 : RÉALISEZ UNE ÉTUDE DE SANTÉ PUBLIQUE - formatage des données pour enregistrement

In [2]:
# PRÉALABLES -------------------------------------------------------------------------------------------------------------------
# Importation des librairies.
# Importations des fichiers.
# Suppression du doublon "Chine".

In [3]:
# Importation des librairies :
import numpy as np
import pandas as pd

In [4]:
# Importation des fichiers
fr_animaux = pd.read_csv("fr_animaux.csv")
fr_cereales = pd.read_csv("fr_cereales.csv")
fr_population = pd.read_csv("fr_population.csv")
fr_sousalimentation = pd.read_csv("fr_sousalimentation.csv",decimal=".",delimiter=",")
fr_vegetaux = pd.read_csv("fr_vegetaux.csv")

In [5]:
# Restriction : suppression des lignes "Chine" dans tous les tableaux :
doublons = ["Chine"]
fr_animaux.drop(fr_animaux.loc[fr_animaux["Zone"].isin(doublons)].index, inplace=True)
fr_cereales.drop(fr_cereales.loc[fr_cereales["Zone"].isin(doublons)].index, inplace=True)
fr_population.drop(fr_population.loc[fr_population["Zone"].isin(doublons)].index, inplace=True)
fr_sousalimentation.drop(fr_sousalimentation.loc[fr_sousalimentation["Zone"].isin(doublons)].index, inplace=True)
fr_vegetaux.drop(fr_vegetaux.loc[fr_vegetaux["Zone"].isin(doublons)].index, inplace=True)

In [6]:
# QUESTION 15 -----------------------------------------------------------------------------------------------------------------
# Créer une table "population" contenant la population de chaque pays en 2013.
# Proposition de clé primaire : [code_pays]

In [14]:
# Projection :
population = fr_population[["Zone","Code zone","Année","Valeur"]]

# Dénomination des colonnes selon les consignes :
population = population.rename(columns={"Code zone":"code_pays", "Zone":"pays", "Année":"annee", "Valeur":"population"})

# Conversion des personnes en personnes :
population["population"] = population["population"] * 1000

# Export vers csv :
# population.to_csv("population.csv", index = False, encoding="ISO-8859-1")
population.head()

,pays,code_pays,annee,population
0,Afghanistan,2,2013,30552000
1,Afrique du Sud,202,2013,52776000
2,Albanie,3,2013,3173000
3,Algérie,4,2013,39208000
4,Allemagne,79,2013,82727000


In [8]:
# QUESTION 16 -----------------------------------------------------------------------------------------------------------------
# Créer une table "dispo_alim" contenant la population de chaque pays en 2013 :
# Proposition de clé primaire : artificielle ou [code_pays,code_produit].

In [16]:
# Création du dataframe dispo_alim :
dispo_alim = pd.concat([fr_animaux, fr_vegetaux])

# Création de la colonne "origin" :
fr_vegetaux["Code Produit"].unique()
produitsVeg = [2511, 2805, 2513, 2514, 2517, 2520, 2531, 2536, 2537, 2542, 2543,
       2745, 2549, 2551, 2557, 2559, 2560, 2561, 2563, 2570, 2571, 2572,
       2573, 2574, 2575, 2577, 2579, 2580, 2586, 2601, 2605, 2611, 2614,
       2615, 2617, 2618, 2619, 2620, 2625, 2630, 2633, 2635, 2640, 2645,
       2655, 2656, 2658, 2680, 2899, 2515, 2516, 2518, 2532, 2533, 2535,
       2534, 2546, 2547, 2555, 2556, 2558, 2562, 2576, 2578, 2582, 2602,
       2612, 2613, 2616, 2641, 2642, 2657, 2659, 2541, 2581]
dispo_alim["origin"] = dispo_alim["Code Produit"].isin(produitsVeg).replace({True:"vegetal", False:"animal"})

dispo_alim = dispo_alim[["Élément","Zone","Code zone","Année","Produit","Code Produit","origin","Valeur"]]

# Restrictions :
dispo_alim1 = dispo_alim.loc[dispo_alim["Élément"]=="Nourriture"].drop(columns=["Élément"]).rename(columns={"Valeur":"dispo_alim_tonnes"})
dispo_alim2 = dispo_alim.loc[dispo_alim["Élément"]=="Disponibilité alimentaire (Kcal/personne/jour)"].drop(columns=["Élément"]).rename(columns={"Valeur":"dispo_alim_kcal_p_j"})
dispo_alim3 = dispo_alim.loc[dispo_alim["Élément"]=="Disponibilité de protéines en quantité (g/personne/jour)"].drop(columns=["Élément"]).rename(columns={"Valeur":"dispo_prot"})
dispo_alim4 = dispo_alim.loc[dispo_alim["Élément"]=="Disponibilité de matière grasse en quantité (g/personne/jour)"].drop(columns=["Élément"]).rename(columns={"Valeur":"dispo_mat_gr"})

# Conversion des milliers de tonnes
dispo_alim1["dispo_alim_tonnes"] = dispo_alim1["dispo_alim_tonnes"] * 1000

# Jointure des différentes restrictions :
dispo_alim = pd.merge(dispo_alim2, dispo_alim1, how="left")
dispo_alim = pd.merge(dispo_alim, dispo_alim3, how="left")
dispo_alim = pd.merge(dispo_alim, dispo_alim4, how="left")

# Dénomination des colonnes selon les consignes :
dispo_alim = dispo_alim.rename(columns={"Code zone":"code_pays", "Zone":"pays", "Année":"annee", "Produit":"produit", "Code Produit":"code_produit"})

# Export vers csv :
# dispo_alim.to_csv("dispo_alim.csv", index = False, encoding="ISO-8859-1")
dispo_alim.head()

,pays,code_pays,annee,produit,code_produit,origin,dispo_alim_kcal_p_j,dispo_alim_tonnes,dispo_prot,dispo_mat_gr
0,Afghanistan,2,2013,Viande de Bovins,2731,animal,27.0,140000.0,1.89,2.10
1,Afghanistan,2,2013,Viande d'Ovins/Caprins,2732,animal,31.0,150000.0,1.91,2.55
2,Afghanistan,2,2013,Viande de Volailles,2734,animal,9.0,75000.0,0.87,0.53
3,Afghanistan,2,2013,"Viande, Autre",2735,animal,1.0,12000.0,0.17,0.06
4,Afghanistan,2,2013,Abats Comestible,2736,animal,5.0,53000.0,0.77,0.20


In [10]:
# QUESTION 17 -----------------------------------------------------------------------------------------------------------------
# Créer une table "dispo_alim" contenant la population de chaque pays en 2013 :
# Proposition de clé primaire : clé artificielle ou [code_pays,code_produit].

In [18]:
# Création du dataframe equilibre_prod sur le modèle fr_alimentation
equilibre_prod = pd.concat([fr_animaux, fr_vegetaux])
equilibre_prod = equilibre_prod[["Zone","Code zone","Année","Élément","Produit","Code Produit","Valeur"]]

# Restrictions :
equilibre_prod1 = equilibre_prod.loc[equilibre_prod["Élément"]=="Disponibilité intérieure"].drop(columns=["Élément"]).rename(columns={"Valeur":"dispo_int"})
equilibre_prod2 = equilibre_prod.loc[equilibre_prod["Élément"]=="Aliments pour animaux"].drop(columns=["Élément"]).rename(columns={"Valeur":"alim_ani"})
equilibre_prod3 = equilibre_prod.loc[equilibre_prod["Élément"]=="Semences"].drop(columns=["Élément"]).rename(columns={"Valeur":"semences"})
equilibre_prod4 = equilibre_prod.loc[equilibre_prod["Élément"]=="Pertes"].drop(columns=["Élément"]).rename(columns={"Valeur":"pertes"})
equilibre_prod5 = equilibre_prod.loc[equilibre_prod["Élément"]=="Traitement"].drop(columns=["Élément"]).rename(columns={"Valeur":"transfo"})
equilibre_prod6 = dispo_alim1.drop(columns=["origin"]).rename(columns={"dispo_alim_tonnes":"nourriture"})
equilibre_prod7 = equilibre_prod.loc[equilibre_prod["Élément"]=="Autres utilisations (non alimentaire)"].drop(columns=["Élément"]).rename(columns={"Valeur":"autres_utilisations"})

# Jointure des différentes restrictions :
equilibre_prod = pd.merge(equilibre_prod1, equilibre_prod2, how="left")
equilibre_prod = pd.merge(equilibre_prod, equilibre_prod3, how="left")
equilibre_prod = pd.merge(equilibre_prod, equilibre_prod4, how="left")
equilibre_prod = pd.merge(equilibre_prod, equilibre_prod5, how="left")
equilibre_prod = pd.merge(equilibre_prod, equilibre_prod6, how="left")
equilibre_prod = pd.merge(equilibre_prod, equilibre_prod7, how="left")

# Conversion des tonnes en milliers de tonnes
equilibre_prod["nourriture"] = equilibre_prod["nourriture"] / 1000

# Dénomination des colonnes selon les consignes :
equilibre_prod = equilibre_prod.rename(columns={"Code zone":"code_pays", "Zone":"pays", "Année":"annee", "Produit":"produit", "Code Produit":"code_produit"})

# Export vers csv :
# equilibre_prod.to_csv("equilibre_prod.csv", index = False, encoding="ISO-8859-1")
equilibre_prod.head()

,pays,code_pays,annee,produit,code_produit,dispo_int,alim_ani,semences,pertes,transfo,nourriture,autres_utilisations
0,Afghanistan,2,2013,Viande de Bovins,2731,140.0,NaN,NaN,NaN,NaN,140.0,NaN
1,Afghanistan,2,2013,Viande d'Ovins/Caprins,2732,150.0,NaN,NaN,NaN,NaN,150.0,NaN
2,Afghanistan,2,2013,Viande de Volailles,2734,75.0,NaN,NaN,NaN,NaN,75.0,NaN
3,Afghanistan,2,2013,"Viande, Autre",2735,12.0,NaN,NaN,NaN,NaN,12.0,NaN
4,Afghanistan,2,2013,Abats Comestible,2736,53.0,NaN,NaN,NaN,NaN,53.0,NaN


In [12]:
# QUESTION 18 -----------------------------------------------------------------------------------------------------------------
# Créer une table "sous_nutrition" contenant le nombre de personnes en sous-nutrition pour chaque pays en 2013 :
# Proposition de clé primaire : clé artificielle ou [code_pays,année].

In [23]:
# Restriction :
sous_nutrition = fr_sousalimentation[["Zone","Code zone","Année","Valeur"]]

# Dénomination des colonnes selon les consignes :
sous_nutrition = sous_nutrition.rename(columns={"Zone":"pays","Code zone":"code_pays","Année":"annee", "Valeur":"nb_personnes"})

# Conversion des millions de personnes en personnes :
sous_nutrition["nb_personnes"] = sous_nutrition["nb_personnes"].replace(to_replace = "<0.1", value = "0")
sous_nutrition["nb_personnes"] = sous_nutrition["nb_personnes"].astype(np.float)
sous_nutrition["nb_personnes"] = sous_nutrition["nb_personnes"] * 1000000

# Export vers csv :
# sous_nutrition.to_csv("sous_nutrition.csv", index = False, encoding="ISO-8859-1")
sous_nutrition.head()

,pays,code_pays,annee,nb_personnes
0,Afghanistan,2,2012-2014,7900000.0
1,Afghanistan,2,2013-2015,8800000.0
2,Afghanistan,2,2014-2016,9600000.0
3,Afghanistan,2,2015-2017,10200000.0
4,Afghanistan,2,2016-2018,10600000.0
...,...,...,...,...
1015,Zimbabwe,181,2012-2014,6600000.0
1016,Zimbabwe,181,2013-2015,7200000.0
1017,Zimbabwe,181,2014-2016,7800000.0
1018,Zimbabwe,181,2015-2017,8200000.0
